In [16]:
# frozenset() 返回一个冻结的集合，冻结后集合不能再添加或删除任何元素。
import time

import numpy as np
import pandas as pd
import math
from Config import *

# 统计各个阶段的时间耗时
times = []

# 统计apriori_gen的组合次数
counts = []
class node:
    def __int__(self):
        self.items = []
        self.key = 0
        self.sup = 0

    def show(self):
        print(self.items, bin(self.key), end=" ")
        try:
            print(self.sup)
        except:
            print()
            pass


def ListToNode(items):
    nodeT = node()
    nodeT.key = 0
    nodeT.items = items
    for index, i in enumerate(defaultSL):  # 进行编码
        if i in items:
            nodeT.key += 1 << (len(defaultSL) - index - 1)
    return nodeT


def keyToList(key):
    ans = []
    global defaultSL
    for i in range(1, len(defaultSL) + 1):
        if key & (1 << (len(defaultSL)) - i) == (1 << (len(defaultSL)) - i):  # 含有第i个
            ans.append(defaultSL[i - 1])
    return ans


# 返回只有单个元素的候选集
def createC1(dataSet):
    '''
        构建初始候选项集的列表，即所有候选项集只包含一个元素，
        C1是大小为1的所有候选项集的集合
    '''
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if [item] not in C1:
                C1.append([item])
    C1.sort()
    # return map( frozenset, C1 )
    # return [var for var in map(frozenset,C1)]
    return [frozenset(var) for var in C1]


def scanD(D, Ck, minSupport):
    '''
        计算Ck中的项集在数据集合D(记录或者transactions)中的支持度,
        返回满足最小支持度的项集的集合，和所有项集支持度信息的字典。
    '''
    print(len(Ck))

    for i in Ck:
        i.sup = 0
        for item in D:
            if i.key & item.key == i.key:  # 与运算判断子集
                i.sup += 1
    ans = []
    length = len(D)
    for i in Ck:
        i.sup = i.sup / length
        if i.sup >= minSupport:
            ans.append(i)
            # i.show()
    return ans


def aprioriGen(Lk, k, L):  # Aprior算法
    '''
        由初始候选项集的集合Lk生成新的生成候选项集，
        k表示生成的新项集中所含有的元素个数
        注意其生成的过程中，首选对每个项集按元素排序，然后每次比较两个项集，只有在前k-1项相同时才将这两项合并。这样做是因为函数并非要两两合并各个集合，那样生成的集合并非都是k+1项的。在限制项数为k+1的前提下，只有在前k-1项相同、最后一项不相同的情况下合并才为所需要的新候选项集。
    '''

    #     print(k)
    left = []  # 存储积最小的两组
    right = []
    lastl = None  # 存储积次小的两组
    lastr = None
    index = None
    minV = 999999999
    minV2 = 999999999
    ans = []
    nowSet = set()
    for i in range(1, math.ceil(k / 2) + 1):
        A = L[i - 1]
        B = L[k - i - 1]
        #         print(len(A[0].items),len(B[0].items) )
        # print(len(A), len(B))
        if len(A) * len(B) < minV:  # 如果比最小的更小,更新最小的,且更新次小的
            index = i
            lastl = left
            lastr = right
            left = A
            right = B
            minV2 = minV
            minV = len(A) * len(B)
        elif len(A) * len(B) < minV2:  # 如果没有比更小的小，但是比次小的小，那么更新次小的
            lastl = A
            lastr = B
            minV2 = len(A) * len(B)
    print('this is ', index)
    counts.append(minV)
    for i in left:
        
        for j in right:
            if i.key & j.key == 0:  # 这俩频繁集合不相交
                t = i.key | j.key
                if t not in nowSet:
                    nowSet.add(t)
                    nodeT = node()
                    nodeT.key = t
                    #                         nodeT.items = keyToList(nodeT.key)  #生成新的items
                    ans.append(nodeT)

            # 在另一组分布里面求可能的组合，然后两个组合取交集， 目前没有好效果
#     returnList = []
#     nowSet2 = set()
#     if k > 3:
#         # print(lastl)
#         left = lastl
#         right = lastr
#         for i in left:
#             for j in right:
#                 if i.key & j.key == 0:  # 这俩频繁集合不相交
#                     t = i.key | j.key
#                     # print(bin(t))
#                     if t not in nowSet2 and t in nowSet :
#                         nowSet2.add(t)
#                         nodeT = node()
#                         nodeT.key = t
#                         #                         nodeT.items = keyToList(nodeT.key)  #生成新的items
#                         returnList.append(nodeT)
#         return returnList
    return ans


def apriori(dataSet, minSupport=0.5):
    """
    该函数为Apriori算法的主函数，按照前述伪代码的逻辑执行。Ck表示项数为k的候选项集，最初的C1通过createC1()函数生成。Lk表示项数为k的频繁项集，supK为其支持度，Lk和supK由scanD()函数通过Ck计算而来。
    :param dataSet:
    :param minSupport:
    :return:
    """
    # C1 = createC1(dataSet)  # 构建初始候选项集C1  [frozenset({1}), frozenset({2}), frozenset({3}), frozenset({4}), frozenset({5})]
    global defaultSL
    global defaultS

    #     print(defaultSL)
    C1 = [ListToNode([i]) for i in defaultSL]

    # D = [set(var) for var in dataSet]  # 集合化数据集

    F1 = scanD(dataSet, C1, minSupport)  # 构建初始的频繁项集，即所有项集只有一个元素

    # print()
    L = [F1]
    k = 2  # 项集应该含有2个元素，所以 k=2

    while (len(L[k - 2]) > 0):
        print("iter is ", k)
        t = time.time()
        Ck = aprioriGen(L[k - 2], k, L)  # 计算候选集
        
        TmpCk = scanD(dataSet[0:int(len(dataSet)/10)], Ck, minSupport *(1/5)) # 使用采样法，进行预处理，减少候选集数目，适当降低minSiupport提高容错度。同时也会降低过滤效果
        t1 = time.time() - t
        print(f'gen coast:{time.time() - t:.8f}s')
        t = time.time()

        Fk = scanD(dataSet, TmpCk, minSupport)  # 筛选最小支持度的频繁项集
        t2 = time.time() - t
        print(f'scan coast:{time.time() - t:.8f}s')
        
        times.append([t1,t2])
        L.append(Fk)  # 将符合最小支持度要求的项集加入L
        k += 1  # 新生成的项集中的元素个数应不断增加
    return L  # 返回所有满足条件的频繁项集的列表，和所有候选项集的支持度信息


def calcConf(freqSet, H, supportData, brl, minConf=0.7):  # 规则生成与评价
    '''
        计算规则的可信度，返回满足最小可信度的规则。
        freqSet(frozenset):频繁项集
        H(frozenset):频繁项集中所有的元素
        supportData(dic):频繁项集中所有元素的支持度
        brl(tuple):满足可信度条件的关联规则
        minConf(float):最小可信度
    '''
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet] / supportData[freqSet - conseq]
        if conf >= minConf:
            print(freqSet - conseq, '-->', conseq, 'conf:', conf)
            brl.append((freqSet - conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH


def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    '''
        对频繁项集中元素超过2的项集进行合并。
        freqSet(frozenset):频繁项集
        H(frozenset):频繁项集中的所有元素，即可以出现在规则右部的元素
        supportData(dict):所有项集的支持度信息
        brl(tuple):生成的规则
    '''
    m = len(H[0])
    if len(freqSet) > m + 1:  # 查看频繁项集是否足够大，以到于移除大小为 m的子集，否则继续生成m+1大小的频繁项集
        Hmp1 = aprioriGen(H, m + 1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)  # 对于新生成的m+1大小的频繁项集，计算新生成的关联规则的右则的集合
        if len(Hmp1) > 1:  # 如果不止一条规则满足要求（新生成的关联规则的右则的集合的大小大于1），进一步递归合并，
            # 这样做的结果就是会有“[1|多]->多”(右边只会是“多”，因为合并的本质是频繁子项集变大，
            # 而calcConf函数的关联结果的右侧就是频繁子项集）的关联结果
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)


def generateRules(L, supportData, minConf=0.7):
    '''
        根据频繁项集和最小可信度生成规则。
        L(list):存储频繁项集
        supportData(dict):存储着所有项集（不仅仅是频繁项集）的支持度
        minConf(float):最小可信度
    '''
    bigRuleList = []
    for i in range(1, len(L)):
        for freqSet in L[i]:  # 对于每一个频繁项集的集合freqSet
            H1 = [frozenset([item]) for item in freqSet]
            if i > 1:  # 如果频繁项集中的元素个数大于2，需要进一步合并，这样做的结果就是会有“[1|多]->多”(右边只会是“多”，
                # 因为合并的本质是频繁子项集变大，而calcConf函数的关联结果的右侧就是频繁子项集），的关联结果
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)

    sorted(bigRuleList)
    return bigRuleList






In [15]:
defaultS = set()  # 存储所有单项的集合
myDat = loadTest()  # 导入数据集
for i in myDat:
    for j in i:
        if j not in defaultS:
            defaultS.add(j)
defaultSL = list(defaultS)
defaultSL = sorted(defaultSL) # 把所有单项计算出来并排序，形成默认顺序，方便后面进行二进制编码
print(defaultSL, len(defaultSL))
Items = []



#对项集进行二进制编码
for items in myDat:
    nodeT = node()
    nodeT.key =0
    nodeT.items = items
    for index,i in enumerate(defaultSL): # 进行编码
        if i in items:
            nodeT.key += 1 << (len(defaultSL) - index -1)
    Items.append(nodeT)
    # print(items, bin(nodeT.key))


['m0e', 'm0p', 'm10e', 'm10t', 'm11b', 'm11c', 'm11e', 'm11r', 'm12f', 'm12s', 'm13f', 'm13s', 'm13y', 'm14w', 'm15w', 'm16p', 'm17w', 'm18o', 'm19e', 'm19p', 'm1b', 'm1f', 'm1s', 'm1x', 'm20k', 'm20n', 'm20u', 'm21a', 'm21n', 'm21s', 'm21v', 'm21y', 'm22d', 'm22g', 'm22m', 'm22p', 'm22u', 'm2f', 'm2s', 'm2y', 'm3g', 'm3n', 'm3w', 'm3y', 'm4f', 'm4t', 'm5a', 'm5l', 'm5n', 'm5p', 'm6f', 'm7c', 'm7w', 'm8b', 'm8n', 'm9g', 'm9k', 'm9n', 'm9p', 'm9w'] 60


In [3]:

twoDif =set() # 计算仅仅两个不同时的二进制集合
length = len(defaultSL)
for i in range(length):
    for j in range(length):
        if i != j:
            t=0
#             print(i,j)
            t = 1<<(i) 
            t+=1<<(j)
#             print(bin(t))
            twoDif.add(t)
# for i in twoDif:
#     print(bin(i))


In [4]:
len(Items)

100

In [17]:
import time
t = time.time()

L = apriori(Items[0:100], 0.22)  # 选择频繁项集
# print(u"频繁项集L：")
# for li in L:
#     for i in li:
#         i.show()
print(f'total coast:{time.time() - t:.8f}s')



60
iter is  2
this is  1
528
gen coast:0.00689101s
461
scan coast:0.01014709s
iter is  3
this is  1
4770
gen coast:0.01689601s
3772
scan coast:0.05832815s
iter is  4
this is  1
26563
gen coast:0.06334710s
19631
scan coast:0.23833179s
iter is  5
this is  1
99579
gen coast:0.41293311s
70373
scan coast:0.81868696s
iter is  6
this is  1
267645
gen coast:0.86875701s
183473
scan coast:2.09409189s
iter is  7
this is  1
537507
gen coast:1.58616281s
360662
scan coast:4.08560109s
iter is  8
this is  1
827845
gen coast:2.37967086s
547039
scan coast:6.18136382s
iter is  9
this is  1
992859
gen coast:3.15468907s
648928
scan coast:7.29879189s
iter is  10
this is  1
933771
gen coast:2.63992286s
605556
scan coast:6.82190204s
iter is  11
this is  1
688415
gen coast:2.12313867s
443946
scan coast:4.95224714s
iter is  12
this is  1
394606
gen coast:0.95422578s
253420
scan coast:2.86109114s
iter is  13
this is  1
172720
gen coast:0.42584276s
110557
scan coast:1.23258233s
iter is  14
this is  1
55876
gen co

In [13]:
counts

[1089,
 124609,
 4368100,
 63266116,
 455438281,
 1809311296,
 4216813969,
 5967099009,
 5206921281,
 2801161476,
 914034289,
 174477681,
 18249984,
 931225,
 18496,
 81,
 0]

In [10]:
for i in L:
    counts.append(len(i))

# 加入多进程进行测试

尽量不要变动了，加入多进程进行测试
数据：5000：0.2

In [5]:
from Config import *
import ray
ray.init()

RayContext(dashboard_url='', python_version='3.7.13', ray_version='1.12.0', ray_commit='f18fc31c7562990955556899090f8e8656b48d2d', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-05-15_14-51-02_924277_71413/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-05-15_14-51-02_924277_71413/sockets/raylet', 'webui_url': '', 'session_dir': '/tmp/ray/session_2022-05-15_14-51-02_924277_71413', 'metrics_export_port': 59059, 'gcs_address': '127.0.0.1:62215', 'address': '127.0.0.1:62215', 'node_id': 'b604a0250bde8d9be2c21fec94f80bef413a6349c405cc65199b9407'})

In [26]:
times = []
import copy
myDat = loadTest()  # 导入数据集


def scanD(D, Ck, minSupport):
    '''
        计算Ck中的项集在数据集合D(记录或者transactions)中的支持度,
        返回满足最小支持度的项集的集合，和所有项集支持度信息的字典。
    '''
    
    print(len(Ck))

    for i in Ck:
        i.sup = 0
        for item in D:
            if i.key & item.key == i.key:  # 与运算判断子集
                i.sup += 1
    ans = []
    length = len(D)
    for i in Ck:
        i.sup = i.sup / length
        if i.sup >= minSupport:
            ans.append(i)
            # i.show()
    return ans


def aprioriGen(Lk, k, L):  # Aprior算法
    '''
        由初始候选项集的集合Lk生成新的生成候选项集，
        k表示生成的新项集中所含有的元素个数
        注意其生成的过程中，首选对每个项集按元素排序，然后每次比较两个项集，只有在前k-1项相同时才将这两项合并。这样做是因为函数并非要两两合并各个集合，那样生成的集合并非都是k+1项的。在限制项数为k+1的前提下，只有在前k-1项相同、最后一项不相同的情况下合并才为所需要的新候选项集。
    '''

    #     print(k)
    left = []  # 存储积最小的两组
    right = []
    lastl = None  # 存储积次小的两组
    lastr = None
    minV = 999999999
    minV2 = 999999999
    ans = []
    nowSet = set()
    for i in range(1, math.ceil(k / 2) + 1):
        A = L[i - 1]
        B = L[k - i - 1]
        #         print(len(A[0].items),len(B[0].items) )
        # print(len(A), len(B))
        if len(A) * len(B) < minV:  # 如果比最小的更小,更新最小的,且更新次小的
            lastl = left
            lastr = right
            left = A
            right = B
            minV2 = minV
            minV = len(A) * len(B)
        elif len(A) * len(B) < minV2:  # 如果没有比更小的小，但是比次小的小，那么更新次小的
            lastl = A
            lastr = B
            minV2 = len(A) * len(B)

    for i in left:
        
        for j in right:
            if i.key & j.key == 0:  # 这俩频繁集合不相交
                t = i.key | j.key
                if t not in nowSet:
                    nowSet.add(t)
                    nodeT = node()
                    nodeT.key = t
                    #                         nodeT.items = keyToList(nodeT.key)  #生成新的items
                    ans.append(nodeT)


    return ans
def apriori(dataSet, minSupport=0.5):
    """
    该函数为Apriori算法的主函数，按照前述伪代码的逻辑执行。Ck表示项数为k的候选项集，最初的C1通过createC1()函数生成。Lk表示项数为k的频繁项集，supK为其支持度，Lk和supK由scanD()函数通过Ck计算而来。
    :param dataSet:
    :param minSupport:
    :return:
    """
    # C1 = createC1(dataSet)  # 构建初始候选项集C1  [frozenset({1}), frozenset({2}), frozenset({3}), frozenset({4}), frozenset({5})]
    global defaultSL
    global defaultS

    #     print(defaultSL)
    C1 = [ListToNode([i]) for i in defaultSL]

    # D = [set(var) for var in dataSet]  # 集合化数据集

    F1 = scanD(dataSet, C1, minSupport)  # 构建初始的频繁项集，即所有项集只有一个元素

    # print()
    L = [F1]
    k = 2  # 项集应该含有2个元素，所以 k=2

    while (len(L[k - 2]) > 0):
        print("iter is ", k)
        t = time.time()
        Ck = aprioriGen(L[k - 2], k, L)  # 计算候选集
        
        TmpCk = scanD(dataSet[0:int(len(dataSet)/10)], Ck, minSupport *(2/3)) # 使用采样法，进行预处理，减少候选集数目，适当降低minSiupport提高容错度。同时也会降低过滤效果
        t1 = time.time() - t
        print(f'gen coast:{time.time() - t:.8f}s')
        t = time.time()

        Fk = total_scan(dataSet, TmpCk, minSupport) # 筛选最小支持度的频繁项集
        t2 = time.time() - t
        print(f'scan coast:{time.time() - t:.8f}s')
        
        times.append([t1,t2])
        L.append(Fk)  # 将符合最小支持度要求的项集加入L
        k += 1  # 新生成的项集中的元素个数应不断增加
    return L  # 返回所有满足条件的频繁项集的列表，和所有候选项集的支持度信息



In [27]:
import time
t = time.time()

L = apriori(Items[0:100], 0.22)  # 选择频繁项集
# print(u"频繁项集L：")
# for li in L:
#     for i in li:
#         i.show()
print(f'total coast:{time.time() - t:.8f}s')


97
iter is  2
528
gen coast:0.00714111s
422
(single_scan pid=71439) 42
(single_scan pid=71439) 42
(single_scan pid=71439) 42
(single_scan pid=71439) 42
(single_scan pid=71439) 42
(single_scan pid=71439) 42
(single_scan pid=71439) 42
(single_scan pid=71439) 42
(single_scan pid=71439) 42
(single_scan pid=71439) 42
scan coast:0.19976211s
scan coast:0.20011091s
iter is  3
4738
gen coast:0.01135397s
3131
(single_scan pid=71439) 1
(single_scan pid=71439) 313
(single_scan pid=71439) 313
(single_scan pid=71439) 313
(single_scan pid=71439) 313
(single_scan pid=71439) 313
(single_scan pid=71439) 313
(single_scan pid=71439) 313
(single_scan pid=71439) 313
(single_scan pid=71439) 313
(single_scan pid=71439) 313
scan coast:0.19196510s
scan coast:0.19270706s
iter is  4
26358
gen coast:0.06324768s
15047
(single_scan pid=71439) 0
(single_scan pid=71439) 1504
(single_scan pid=71439) 1504
(single_scan pid=71439) 1504
(single_scan pid=71440) 1504
(single_scan pid=71440) 1504
(single_scan pid=71440) 1504


(single_scan pid=71440) 4
(single_scan pid=71436) 65
(single_scan pid=71436) 8
(single_scan pid=71436) 4
(single_scan pid=71436) 4
scan coast:0.18021393s
scan coast:0.18056083s
total coast:30.09993410s
(single_scan pid=71440) 4
(single_scan pid=71440) 4
(single_scan pid=71440) 4
(single_scan pid=71440) 4
(single_scan pid=71440) 4
(single_scan pid=71440) 4
(single_scan pid=71440) 2


In [28]:
for i in L:
    print(len(i))

33
350
2071
7903
21266
42471
64904
77238
72158
52926
30233
13209
4272
965
136
9
0
